In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings # --> Filtro de advertencias para ignorarlas y mantener el código limpio de mensajes no deseados
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)

----------------

# ETL

In [2]:
df1 = pd.read_json('../Datasets/Google maps/Google Maps/data-sitios_comercio_atributes/1.json', lines= True)

In [3]:
df2 = pd.read_json('../Datasets/Google maps/Google Maps/data-sitios_comercio_atributes/2.json', lines=True)

In [4]:
df3 = pd.read_json('../Datasets/Google maps/Google Maps/data-sitios_comercio_atributes/3.json', lines=True)

In [5]:
df4 = pd.read_json('../Datasets/Google maps/Google Maps/data-sitios_comercio_atributes/4.json', lines=True)

In [6]:
df5 = pd.read_json('../Datasets/Google maps/Google Maps/data-sitios_comercio_atributes/5.json', lines=True)

In [7]:
df6 = pd.read_json('../Datasets/Google maps/Google Maps/data-sitios_comercio_atributes/6.json', lines=True)

In [8]:
df7 = pd.read_json('../Datasets/Google maps/Google Maps/data-sitios_comercio_atributes/7.json', lines=True)

In [9]:
df8 = pd.read_json('../Datasets/Google maps/Google Maps/data-sitios_comercio_atributes/8.json', lines=True)

In [10]:
df9 = pd.read_json('../Datasets/Google maps/Google Maps/data-sitios_comercio_atributes/9.json', lines=True)

In [11]:
df10 = pd.read_json('../Datasets/Google maps/Google Maps/data-sitios_comercio_atributes/10.json', lines=True)

In [12]:
df11 = pd.read_json('../Datasets/Google maps/Google Maps/data-sitios_comercio_atributes/11.json', lines=True)

In [13]:
dataframes = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11]

In [14]:
df = pd.concat(dataframes, axis=0, ignore_index=True)

In [15]:
df_philadelphia = df[df['address'].str.contains('philadelphia', case=False, na=False)]

In [173]:
#df_philadelphia.to_csv('df_metadata_sitios_philadelphia.csv', index=False)


In [174]:
#df_philadelphia = pd.read_csv('df_metadata_sitios_philadelphia.csv')

## Caracteristicas y Nulos

In [16]:
df_philadelphia.shape

(15131, 15)

In [17]:
df_philadelphia.columns

Index(['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC',
       'state', 'relative_results', 'url'],
      dtype='object')

## String vacios

In [18]:
df_philadelphia.apply(lambda x: (x == "").sum())

name                0
address             0
gmap_id             0
description         0
latitude            0
longitude           0
category            0
avg_rating          0
num_of_reviews      0
price               0
hours               0
MISC                0
state               0
relative_results    0
url                 0
dtype: int64

In [19]:
df_philadelphia.isnull().sum()

name                    0
address                 0
gmap_id                 0
description         13523
latitude                0
longitude               0
category               99
avg_rating              0
num_of_reviews          0
price               13312
hours                3990
MISC                 3537
state                3817
relative_results     1159
url                     0
dtype: int64

In [20]:
# Eliminamos nulos de categorias ya que esos datos no nos van a servir
df_philadelphia.dropna(subset=['category'], inplace=True)

## Duplicados

In [21]:
# Identificar columnas con diccionarios o listas
columns_with_dicts_or_lists = df_philadelphia.applymap(lambda x: isinstance(x, (dict, list))).any()

# Filtrar los nombres de las columnas que contienen diccionarios o listas
columns_with_dicts_or_lists = columns_with_dicts_or_lists[columns_with_dicts_or_lists].index.tolist()

# Mostrar las columnas que contienen diccionarios o listas
print(f"Columnas con diccionarios o listas: {columns_with_dicts_or_lists}")



Columnas con diccionarios o listas: ['category', 'hours', 'MISC', 'relative_results']


In [22]:
# Especificar las columnas a excluir
columns_to_exclude = ['category', 'hours', 'relative_results', 'MISC']

# Seleccionar las columnas a incluir en la búsqueda de duplicados
columns_to_check = df_philadelphia.columns.difference(columns_to_exclude)

# Contar los duplicados por columna
for col in columns_to_check:
    num_duplicados_col = df_philadelphia[col].duplicated().sum()
    print(f" '{col}': {num_duplicados_col}")


 'address': 136
 'avg_rating': 14991
 'description': 13987
 'gmap_id': 118
 'latitude': 799
 'longitude': 865
 'name': 1441
 'num_of_reviews': 14649
 'price': 15024
 'state': 14441
 'url': 118


In [23]:
# Especificar las columnas a excluir
columns_to_exclude = ['category', 'hours', 'relative_results', 'MISC']

# Seleccionar las columnas a incluir en la búsqueda de duplicados
columns_to_check = df_philadelphia.columns.difference(columns_to_exclude)

# Verificar si existen duplicados globales basados en las columnas seleccionadas
duplicados_globales = df_philadelphia.duplicated(subset=columns_to_check, keep=False)

# Contar cuántos duplicados globales hay
total_duplicados_globales = duplicados_globales.sum()

print(f"Total de duplicados globales: {total_duplicados_globales}")


Total de duplicados globales: 236


In [25]:
# Eliminar los duplicados globales basados en las columnas seleccionadas
df_philadelphia_sin_duplicados = df_philadelphia.drop_duplicates(subset=columns_to_check, keep='first')

# Verificar cuántas filas quedan después de eliminar los duplicados
print(f"Total de filas después de eliminar duplicados: {len(df_philadelphia_sin_duplicados)}")


Total de filas después de eliminar duplicados: 14914


In [27]:
df_philadelphia.dtypes

name                 object
address              object
gmap_id              object
description          object
latitude            float64
longitude           float64
category             object
avg_rating          float64
num_of_reviews        int64
price                object
hours                object
MISC                 object
state                object
relative_results     object
url                  object
dtype: object

In [28]:
# Eliminacion de espacios al inicio y al final de la cadena de string
df_philadelphia['name'] = df_philadelphia['name'].astype(str)
df_philadelphia['name'] = df_philadelphia['name'].str.strip()

In [29]:
df_philadelphia['address'] = df_philadelphia['address'].astype(str)
df_philadelphia['address'] = df_philadelphia['address'].str.strip()

In [30]:
df_philadelphia['description']

68         Bubble tea shop offering hot & cold versions, plus a variety of flavors & a simple snack menu.
69                                                                                                   None
70                                                                                                   None
970                                                                                                  None
972                                                                                                  None
                                                        ...                                              
3024233                                                                                              None
3024244                                                                                              None
3024981                Long-running chain serving signature breakfast items & a variety of coffee drinks.
3024992                                       

In [31]:
df_philadelphia['description'] = df_philadelphia['description'].astype(str)
df_philadelphia['description'] = df_philadelphia['description'].str.strip()

In [32]:
df_philadelphia['category']

68                                                                                                          [Restaurant]
69                                                                                   [Bicycle Shop, Bicycle repair shop]
70                                                [Bicycle Shop, Bicycle repair shop, Coffee shop, Sporting goods store]
970                                                                                                   [Auto repair shop]
972                                                      [Attorney, Criminal justice attorney, Personal injury attorney]
                                                               ...                                                      
3024233                                                      [Culinary school, Bakery, College, Pastry shop, Restaurant]
3024244                                                                                                   [Dance school]
3024981    [Coffee shop, Bagel s

In [33]:
df_philadelphia['category'] = df_philadelphia['category'].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))




In [34]:
df_philadelphia['category'] = df_philadelphia['category'].astype(str)
df_philadelphia['category'] = df_philadelphia['category'].str.strip()

In [35]:
df_philadelphia['category']

68                                                                                                          Restaurant
69                                                                                   Bicycle Shop, Bicycle repair shop
70                                                Bicycle Shop, Bicycle repair shop, Coffee shop, Sporting goods store
970                                                                                                   Auto repair shop
972                                                      Attorney, Criminal justice attorney, Personal injury attorney
                                                              ...                                                     
3024233                                                      Culinary school, Bakery, College, Pastry shop, Restaurant
3024244                                                                                                   Dance school
3024981    Coffee shop, Bagel shop, Bakery, Brea

In [36]:
df_philadelphia['price'].unique()

array([None, '$', '$$', '$$$', '$$$$', '₩₩', '₩₩₩', '₩'], dtype=object)

In [37]:
df_philadelphia.drop(columns=['price'], inplace= True)

In [38]:
pd.set_option('display.max_colwidth', None)

In [40]:
df_philadelphia['relative_results'] = df_philadelphia['relative_results'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)


In [41]:
df_philadelphia['relative_results']

68                                                0x89c6c62a40f4b733:0x73bfeec1898f272b, 0x89c6d71eb1190821:0x31bebc9b0a087ebc, 0x89c6c62a8b1da2cd:0xa1f0f7939020c2e1, 0x89c6c62a0572343f:0xf225afb6cd0ac40f
69          0x89c6c623342d6197:0x70233738cd92d9b, 0x89c6c63ade202695:0x242762f97f9a5d35, 0x89c6c61ee8a3f66f:0x4c05fe1e923a7bf4, 0x89c6c60f7abc3071:0xe1a74583216cf069, 0x89c6c640b99b6e73:0x8705b80be823f27f
70                                                                                                                              0x89c6c7d1a79d4103:0x2476b0fe30eb0871, 0x89c6c7d84645270d:0x34c1faf6ca84b9a9
970        0x89c6c61d271e254b:0xe6a43cb5d72adb16, 0x89c6c61ef5ea56b7:0x21ea24499ee00866, 0x89c6c6192406d463:0x53baca868c0ac8a7, 0x89c6c7937d6a7a79:0x120a979f70d78e96, 0x89c6c6a21371b7f7:0x817b30ad4395fc04
972         0x89c6c631c4409039:0xe0ad1e9d5b7a2846, 0x89c6cc751add9c0b:0x6392e65505c055db, 0x89c6c6255756e4d9:0x33f05f34acffd98, 0x89c6c62e270a51cb:0x282b37513cad2487, 0x89c6c631ed9

In [42]:
df_philadelphia['url']

68         https://www.google.com/maps/place//data=!4m2!3m1!1s0x89c6c89efcaed69d:0xded973f6033e7dba?authuser=-1&hl=en&gl=us
69         https://www.google.com/maps/place//data=!4m2!3m1!1s0x89c6c63c8cd87141:0x54d0d283872eecbb?authuser=-1&hl=en&gl=us
70         https://www.google.com/maps/place//data=!4m2!3m1!1s0x89c6c715d1821fe3:0x9cfa8308c0ce2289?authuser=-1&hl=en&gl=us
970        https://www.google.com/maps/place//data=!4m2!3m1!1s0x89c6c61c782d4239:0xee40c6ec66db8e92?authuser=-1&hl=en&gl=us
972        https://www.google.com/maps/place//data=!4m2!3m1!1s0x89c6c6316dcbf9cf:0xa14cd710255db485?authuser=-1&hl=en&gl=us
                                                                 ...                                                       
3024233    https://www.google.com/maps/place//data=!4m2!3m1!1s0x89c6c6fa06c8981b:0x66ec244795b4378f?authuser=-1&hl=en&gl=us
3024244    https://www.google.com/maps/place//data=!4m2!3m1!1s0x89c6c6eeed5aad33:0xfd73f1ebc99b9da7?authuser=-1&hl=en&gl=us
3024981 

In [43]:
# Especificar las columnas a excluir
columns_to_exclude = ['hours', 'MISC']

# Seleccionar las columnas a incluir en la búsqueda de duplicados
columns_to_check = df_philadelphia.columns.difference(columns_to_exclude)

# Verificar si existen duplicados globales basados en las columnas seleccionadas
duplicados_globales = df_philadelphia.duplicated(subset=columns_to_check, keep=False)

# Contar cuántos duplicados globales hay
total_duplicados_globales = duplicados_globales.sum()

print(f"Total de duplicados globales: {total_duplicados_globales}")

Total de duplicados globales: 236


In [46]:
# Eliminar duplicados globales basados en las columnas seleccionadas
df_philadelphia_sin_duplicados = df_philadelphia.drop_duplicates(subset=columns_to_check, keep='first', inplace=True)




In [47]:
# Renombrar las columnas 'name' y 'categories'
df_philadelphia.rename(columns={'name': 'name_business', 'categories': 'categories_google'}, inplace=True)

In [48]:
df_philadelphia.columns

Index(['name_business', 'address', 'gmap_id', 'description', 'latitude',
       'longitude', 'category', 'avg_rating', 'num_of_reviews', 'hours',
       'MISC', 'state', 'relative_results', 'url'],
      dtype='object')

In [51]:
df_philadelphia['name_business']

68                                      Vivi Bubble Tea
69                                 Kater Street Bicycle
70               Cadence Cycling and Cafe - Center City
970                      Rizzio's General Auto Rpr Shop
972                                Lipschutz & Friedman
                               ...                     
3024233    The Restaurant School at Walnut Hill College
3024244                      Take the Lead Dance Studio
3024981                                         Dunkin'
3024992                                   Reuben's Marc
3025006                                        Steak 48
Name: name_business, Length: 14914, dtype: object

In [52]:
df_philadelphia['address']

68                                        Vivi Bubble Tea, 701 S 5th St, Philadelphia, PA 19147
69                                  Kater Street Bicycle, 609 S 16th St, Philadelphia, PA 19146
70                Cadence Cycling and Cafe - Center City, 201 S 25th St, Philadelphia, PA 19103
970                         Rizzio's General Auto Rpr Shop, 916 Reed St, Philadelphia, PA 19147
972           Lipschutz & Friedman, 1800 John F Kennedy Blvd 11th floor, Philadelphia, PA 19103
                                                   ...                                         
3024233    The Restaurant School at Walnut Hill College, 4207 Walnut St, Philadelphia, PA 19104
3024244                        Take the Lead Dance Studio, 4701 Pine St, Philadelphia, PA 19143
3024981                                       Dunkin', 7000 Chester Ave, Philadelphia, PA 19142
3024992                                 Reuben's Marc, 8131 Stenton Ave, Philadelphia, PA 19150
3025006                                 

In [53]:
# Definir una función para procesar cada fila
def process_address(address):
    # Dividir por comas
    parts = [part.strip() for part in address.split(',')]
    
    # Mantener solo la segunda parte como dirección y el código postal (último valor)
    street_address = parts[1] if len(parts) > 1 else None
    postal_code = parts[-1].replace('PA', '').strip() if len(parts) > 1 else None
    
    return street_address, postal_code

# Aplicar la función a la columna 'address'
df_philadelphia[['street_address', 'postal_code']] = df_philadelphia['address'].apply(process_address).apply(pd.Series)

# Eliminar la columna original y la columna 'Philadelphia'
df_philadelphia.drop(columns=['address'], inplace=True)

In [54]:
df_philadelphia.columns

Index(['name_business', 'gmap_id', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'hours', 'MISC', 'state',
       'relative_results', 'url', 'street_address', 'postal_code'],
      dtype='object')